# PROMOTION BUMP ASSIGNMENT
date: 2024-09-19  
Analist: Botan Fırat Bulut

## IMPORTS AND SETUP

In [19]:
# Imports:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Ignoner harmless warnings:
warnings.filterwarnings('ignore')

# Plot rcParams:
plt.rcParams['figure.figsize'] = (15, 7)
plt.rcParams['figure.dpi'] = 144
plt.rcParams['figure.titlesize'] = 22
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.titleweight'] = 'bold'
plt.style.use('ggplot')

### Data = assignment4.1a.csv
The data contains daily sales of sample of items in several stores on  
a specific time frame. Negative sale quantities represents returns.  
Each row represents a sale (or return) activity for an item in a store at  
a specific day. If a store-item combination has no observation in a  
certain day you can assume there is no sales for that item at that   
store at that day.

In [20]:
# Read assignment4.1a.csv:
df1a = pd.read_csv('./assignment4.1a.csv')
df1a.head()

,Date,StoreCode,ProductCode,SalesQuantity
0,2015-01-01,8,9,-1
1,2015-01-01,131,9,1
2,2015-01-01,144,9,2
3,2015-01-01,203,9,2
4,2015-01-01,256,9,0


In [21]:
# df1a information:
df1a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873618 entries, 0 to 1873617
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Date           object
 1   StoreCode      int64 
 2   ProductCode    int64 
 3   SalesQuantity  int64 
dtypes: int64(3), object(1)
memory usage: 57.2+ MB


In [22]:
# df1a null values:
df1a.isnull().sum()

Date             0
StoreCode        0
ProductCode      0
SalesQuantity    0
dtype: int64

The assignment4.1a.csv contains 3 columns with 1,873,618 records.  
It does not contain any missing values.

In [ ]:
df_promotion = 